At this point in the experiment, we now implement the NVAR onto the measured and integrated data to produce weight matrices for each protocol.

In [1]:
# Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from NVAR1 import *
from NVAR_objective import prepare_data

In [9]:
# The current protocols to assess.
protocols = [17, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]

TRAIN = 96000 # Approximately optimal value from hyperparameter tuning.
TEST = 30000  # Arbitrary. Not required for this program.
dt = 1

# The filepaths for the NORMALISED signals.
host_dict = os.path.dirname(os.getcwd())
filepath = os.path.join(host_dict, "data", "Normalised_Signals")

# The filepath for the results' directory.
result_dict = os.path.join(os.getcwd(), "results", "model_weights")
# result_dict = os.path.join(os.getcwd(), "results", "model_weights_full_length")

print(filepath)
print(result_dict)

/home/dafydd/PycharmProjects/Simulating-Neuronic-Oscillations-with-Reservoir-Computing/data/Normalised_Signals
/home/dafydd/PycharmProjects/Simulating-Neuronic-Oscillations-with-Reservoir-Computing/init_NGRC/results/model_weights


In [10]:
for i in protocols:
        # Locating the filepaths for extraction.
    measured_filepath = os.path.join(filepath, f"M_NProtocol_{i}")
    integrated_filepath = os.path.join(filepath, f"I_NProtocol_{i}")
        
    measured = pd.read_csv(measured_filepath, header=0, dtype="float64", delimiter=",").values
    integrated = pd.read_csv(integrated_filepath, header=0, dtype="float64", delimiter=",").values
        
    print(f"Measured Shape = {measured.shape}")
    print(f"Integrated Shape = {integrated.shape}")
    print("\n"*2)

Measured Shape = (211001, 2)
Integrated Shape = (102728, 2)



Measured Shape = (211001, 2)
Integrated Shape = (101598, 2)


Measured Shape = (211001, 2)
Integrated Shape = (101912, 2)



Measured Shape = (211001, 2)
Integrated Shape = (101485, 2)


Measured Shape = (211001, 2)
Integrated Shape = (102107, 2)



Measured Shape = (211001, 2)
Integrated Shape = (102224, 2)


Measured Shape = (201001, 2)
Integrated Shape = (96554, 2)



Measured Shape = (201001, 2)
Integrated Shape = (96774, 2)


Measured Shape = (201001, 2)
Integrated Shape = (96709, 2)



Measured Shape = (201001, 2)
Integrated Shape = (96948, 2)


Measured Shape = (201001, 2)
Integrated Shape = (97164, 2)


In [13]:
for i in protocols:
    
    # Initialise autoregressor.
    nvar = NVAR(delay=2, strides=3, order=2)
    
    # Locating the filepaths for extraction.
    measured_filepath = os.path.join(filepath, f"M_NProtocol_{i}")
    integrated_filepath = os.path.join(filepath, f"I_NProtocol_{i}")
    
    # Importing the voltage and current data.
    measured = pd.read_csv(measured_filepath, header=0, dtype="float64", delimiter=",").values
    integrated = pd.read_csv(integrated_filepath, header=0, dtype="float64", delimiter=",").values
    
    # Splitting the dataset as necessary. 
    mx_train = measured[0:TRAIN, :]
    my_train = measured[1:TRAIN + dt, -1].reshape(-1, 1)
    
    ix_train = integrated[0:TRAIN, :]
    iy_train = integrated[1:TRAIN + dt, -1].reshape(-1, 1)
    
    
    # Perform fitting and training, extract the weights from it.
    M_features = nvar.fit(mx_train)
    M_weights = np.squeeze(nvar.train(M_features, my_train, ridge=0, bias=True))
    
    I_features = nvar.fit(ix_train)
    I_weights = np.squeeze(nvar.train(I_features, my_train, ridge=0, bias=True))
    
    df = pd.DataFrame({
            "Measured": M_weights,
            "Integrated": I_weights,
            "Difference": np.abs(M_weights) - np.abs(I_weights)
    })
    
    filename = os.path.join(result_dict, f"NProtocol_{i}_weights.csv")
    df.to_csv(filename, index=False)

# Weights Comparative Analysis

In [2]:
os.getcwd()

'/home/dafydd/PycharmProjects/Simulating-Neuronic-Oscillations-with-Reservoir-Computing/init_NGRC'

In [ ]:
# Assuming your CSV files are in a directory named 'csv_data'
weights_dict = os.path.join(os.getcwd(), "results", "model_weights") 

# Initialize an empty list to store DataFrame objects
dataframes = []

# Read each CSV file and append the DataFrame to the list
for filename in os.listdir(weights_dict):
    if filename.endswith('.csv'):
        filepath = os.path.join(weights_dict, filename)
        df = pd.read_csv(filepath, header=1, names=['Measured', 'Integrated', 'Difference'], dtype="float32")
        dataframes.append(df)

# Plot a grouped bar chart for each CSV file
for i, df in enumerate(dataframes):
    print(df.iloc[:, 2].values[11])